In [15]:
import torch
import torch.nn as nn


config = {
    'train_file_path': 'dataset/train.csv',
    'test_file_path': 'dataset/test.csv',
    'embedding_path': 'dataset/sgns.weibo.word.bz2',
    'train_val_ratio': 0.1,
    'vocab_size': 30000,
    'batch_size': 64,
    'num_epochs': 10,
    'learning_rate': 1e-3,
    'logging_step': 300,
    'seed': 10003
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

10003

In [62]:
# train.csv 四列 id, label, label_desc, sentence
from collections import Counter
from tqdm import tqdm
import jieba
def get_vocab(config):
    token_counter = Counter()
    with open(config['train_file_path'], 'r', encoding='utf8') as f:
        lines = f.readlines()
        for line in tqdm(lines, desc='Counting tokens', total=len(lines)):
            sent = line.split(',')[-1].strip()
            sent_cut = list(jieba.cut(sent))
            token_counter.update(sent_cut)
            # token_counter {'我': 2,'是': 5}
    
    vocab = set(token for token, _ in token_counter.most_common(config['vocab_size']))
    return vocab

In [63]:
vocab = get_vocab(config)

Counting tokens: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 53361/53361 [00:05<00:00, 9519.25it/s]


In [12]:
import bz2

with bz2.open(config['embedding_path']) as f:
    token_vector = f.readlines()

In [14]:
for i, line in enumerate(token_vector):
    if i == 10:
        line = line.split()
        print(line[0].decode('utf-8'))
        # print(line[1:])
        print(len(line[1:]))
        break

是
300


In [65]:
# 将 词典（vocab） 中的token 转化为 词向量
# token -> embedding 
# token -> id

# '是' <-> 10 <-> 300d Vector

def get_embedding(vocab):
    token2embedding ={}

    with bz2.open('dataset/sgns.weibo.word.bz2') as f:
        token_vector = f.readlines()

        meta_info = token_vector[0].split()
        print(f'{meta_info[0]} tokens in embedding file in total, vector size is {meta_info[1]}')

        # sgns.weibo.word.bz2 从第二行开始，每一行是 'token embedding' 的形式
        # '我' 0.88383 0.22222 *300
        for line in tqdm(token_vector[1:]):
            line = line.split()
            token = line[0].decode('utf8')

            vector = line[1:]

            if token in vocab:
                token2embedding[token] = [float(num) for num in vector]

        # enumerate(, [start])
        token2id = {token: idx for idx, token in enumerate(token2embedding.keys(), 4)}
        id2embedding = {token2id[token]: embedding for token, embedding in token2embedding.items()}

        PAD, UNK, BOS, EOS = '<pad>', '<unk>', '<bos>', '<eos>'

        token2id[PAD] = 0
        token2id[UNK] = 1
        token2id[BOS] = 2
        token2id[EOS] = 3

        id2embedding[0] = [.0] * int(meta_info[1])
        id2embedding[1] = [.0] * int(meta_info[1])

        id2embedding[2] = np.random.random(int(meta_info[1])).tolist()
        id2embedding[3] = np.random.random(int(meta_info[1])).tolist()

        emb_mat = [id2embedding[idx] for idx in range(len(id2embedding))]

        return torch.tensor(emb_mat, dtype=torch.float), token2id, len(vocab)+4

In [66]:
emb_mat, token2id, config['vocab_size'] = get_embedding(vocab)
print(token2id['你'])

b'195202' tokens in embedding file in total, vector size is b'300'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195202/195202 [00:04<00:00, 41941.50it/s]

31


In [67]:
def tokenizer(sent, token2id):
    ids = [token2id.get(token, 1) for token in jieba.cut(sent)]
    return ids

In [68]:
import pandas as pd
from collections import defaultdict
def read_data(config, token2id, mode='train'):
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    if mode == 'train':
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(config['train_val_ratio'] * len(data_df))
    
    else:
        X_test, y_test = defaultdict(list), []

    for i, row in tqdm(data_df.iterrows(), desc=f'Preprocesing {mode} data', total=len(data_df)):
        label=row[1] if mode == 'train' else 0
        sentence = row[-1]
        inputs = tokenizer(sentence, token2id)
        if mode == 'train':
            if i < num_val:
                X_val['input_ids'].append(inputs)
                y_val.append(label)
            else:
                X_train['input_ids'].append(inputs)
                y_train.append(label)
        
        else:
            X_test['input_ids'].append(inputs)
            y_test.append(label)

    if mode == 'train':
        label2id = {label: i for i, label in enumerate(np.unique(y_train))}
        id2label = {i: label for label, i in label2id.items()}

        y_train = torch.tensor([label2id[label] for label in y_train], dtype=torch.long)
        y_val = torch.tensor([label2id[label] for label in y_val], dtype=torch.long)

        return X_train, y_train, X_val, y_val, label2id, id2label
    
    else:
        y_test = torch.tensor(y_test, dtype=torch.long)
        return X_test, y_test

In [69]:
X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, token2id, mode='train')
X_test, y_test = read_data(config, token2id, mode='test')

Preprocesing test data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5732.21it/s]


In [70]:
from torch.utils.data import Dataset
class TNEWSDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __getitem__(self, idx):
        return {
            'input_ids': self.x['input_ids'][idx],
            'label': self.y[idx]
        }
    
    def __len__(self):
        return self.y.size(0)

In [71]:
def collete_fn(examples):
    input_ids_list =[]
    labels = []
    for example in examples:
        input_ids_list.append(example['input_ids'])
        labels.append(example['label'])
    
    # 1.找到 input_ids_list 中最长的句子
    max_length = max(len(input_ids) for input_ids in input_ids_list)

    # 2. 定义一个Tensor
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)

    for i, input_ids in enumerate(input_ids_list):
        # 3.得到当前句子长度
        seq_len = len(input_ids)
        input_ids_tensor[i, :seq_len] = torch.tensor(input_ids, dtype=torch.long)

    return {
        'input_ids': input_ids_tensor,
        'label': torch.tensor(labels, dtype=torch.long)
    }

In [72]:
from torch.utils.data import DataLoader

def build_dataloader(config, vocab):
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, token2id, mode='train')
    X_test, y_test = read_data(config, token2id, mode='test')

    train_dataset = TNEWSDataset(X_train, y_train)
    val_dataset = TNEWSDataset(X_val, y_val)
    test_dataset = TNEWSDataset(X_test, y_test)
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=True, collate_fn=collete_fn)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collete_fn)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collete_fn)

    return id2label, train_dataloader, val_dataloader, test_dataloader

In [73]:
id2label, train_dataloader, val_dataloader, test_dataloader = build_dataloader(config, vocab)

Preprocesing test data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5748.83it/s]


In [40]:
for batch in train_dataloader:
    print(len(batch['input_ids']))
    print(batch)
    break

64
{'input_ids': tensor([[ 2956,    19,  9918,  ...,     0,     0,     0],
        [ 3552,     1,  3111,  ...,     0,     0,     0],
        [17870,     1,  6017,  ...,     0,     0,     0],
        ...,
        [10721,   780,  8378,  ...,     0,     0,     0],
        [   16, 12265,   770,  ...,     0,     0,     0],
        [24386,  3763,  4647,  ...,     0,     0,     0]]), 'label': tensor([ 2,  4,  1,  7, 14,  3,  3,  2, 14,  3,  6,  2,  2,  8, 14,  8,  8, 14,
         1, 14, 13, 14, 14, 10,  4,  6, 13, 11, 11,  1,  7,  6,  8, 10,  2,  9,
        13,  4,  3,  1,  3,  7,  7, 14,  6,  9,  4,  9, 10, 13, 10,  8, 14,  4,
         9, 11,  5,  8,  6,  1,  8, 13, 10, 11])}


In [74]:
model_config = {
    'embedding_pretrained' : emb_mat,
    'num_filters' : 256,
    'emb_size' : emb_mat.shape[1],
    'dropout' : 0.3,
    'filter_sizes' : [2,3,5],
    'num_classes' : len(label2id)
}

In [75]:
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(config['embedding_pretrained'], freeze=True)

        self.convs = nn.ModuleList([nn.Conv2d(1, config['num_filters'], (k, config['emb_size'])) for k in config['filter_sizes']])

        self.dropout = nn.Dropout(config['dropout'])

        # 变换维度，得到logits
        self.fc = nn.Linear(len(config['filter_sizes'] * config['num_filters']), config['num_classes'])

    def convs_and_pool(self, x, conv):

        # x [batch_size, out_channels, seq_len_out, 1]
        # x [batch_size, out_channels, seq_len_out]
        x = F.relu(conv(x)).squeeze(3)

        # x (batch_size, out_channels, 1)
        # x (batch_size, out_channels)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, input_ids=None, label=None):
        # out [batch_size, seq_len, embedding_dim]
        out = self.embedding(input_ids)
        
        # H: seq_len; W:embedding_dim
        # out [batch_size, 1, seq_len, embedding_dim]
        out = out.unsqueeze(1)

        # (batch_size, out_channels)
        out = torch.cat([self.convs_and_pool(out, conv) for conv in self.convs], 1)

        out = self.dropout(out)

        out = self.fc(out)

        output = (out, )

        if label is not None: # 训练集用
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(out, label)
            output = (loss, ) + output

        # train output (loss, out)
        # test output (out)
        return output

In [76]:
model = Model(model_config)

In [77]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss =0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))
    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['label'])
            batch = {item: value.to(config['device']) for item, value in batch.items()}

            # val output (loss, out)
            loss, logits = model(**batch)[:2]
            val_loss += loss.item()

            preds.append(logits.argmax(dim=-1).detach().cpu())
    
    avg_val_loss = val_loss/len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()

    f1 = f1_score(labels, preds, average='macro')
    return avg_val_loss, f1
            

In [78]:
from torch.optim import AdamW
from tqdm import trange
def train(model, config, id2label, train_dataloader, val_dataloader):
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])
    model.to(config['device'])
    epoches_iterator = trange(config['num_epochs'])
    
    # 1 epoch * 200个 batch * 10 = global_step
    global_step = 0
    train_loss = 0.
    logging_loss = 0.

    for epoch in epoches_iterator:
        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        for batch in train_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}

            # train output (loss, out)
            loss = model(**batch)[0]

            model.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss
            global_step += 1

            if global_step % config['logging_step'] == 0:
                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss

                avg_val_loss, f1 = evaluation(config, model, val_dataloader)
                print_log = f'>>> training loss: {print_train_loss:.4f}, valid loss: {avg_val_loss:.4f}, valid f1 score: {f1:.4f}'
                print(print_log)
                model.train()

    return model

In [79]:
best_model = train(model, config, id2label, train_dataloader, val_dataloader)

Training:  37%|███████████████████████████████████████████▊                                                                         | 281/751 [00:01<00:02, 208.60it/s]

Evaluation:   0%|                                                                                                                               | 0/84 [00:00<?, ?it/s]

Evaluation:   1%|█▍                                                                                                                     | 1/84 [00:00<00:17,  4.65it/s]

Evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 175.86it/s]

Training:  43%|██████████████████████████████████████████████████▎                                                                  | 323/751 [00:02<00:04, 103.20it/s]

>>> training loss: 1.6934, valid loss: 1.4309, valid f1 score: 0.4808



Training:  79%|████████████████████████████████████████████████████████████████████████████████████████████▍                        | 593/751 [00:03<00:00, 240.82it/s]

Evaluation:   0%|                                                                                                                               | 0/84 [00:00<?, ?it/s]

Evaluation:   1%|█▍                                                                                                                     | 1/84 [00:00<00:17,  4.62it/s]

Evaluation:  50%|██████████████████████████████████████████████████████████▌                                                          | 42/84 [00:00<00:00, 165.63it/s]

Evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 163.78it/s]

Training:  82%|█████████████████████████████████████████████████████████████████████████████████████████████████                     | 618/751 [00:04<00:0

>>> training loss: 0.0000, valid loss: 1.4249, valid f1 score: 0.4893



Training:  19%|█████████████████████▉                                                                                               | 141/751 [00:01<00:03, 167.76it/s]

Evaluation:   0%|                                                                                                                               | 0/84 [00:00<?, ?it/s]

Evaluation:   1%|█▍                                                                                                                     | 1/84 [00:00<00:22,  3.65it/s]

Evaluation:  51%|███████████████████████████████████████████████████████████▉                                                         | 43/84 [00:00<00:00, 146.22it/s]

Evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 146.48it/s]

Training:  24%|████████████████████████████▏                                                                                         | 179/751 [00:01<00:0

...

>>> training loss: 0.0000, valid loss: 1.8243, valid f1 score: 0.5177



Training:  56%|█████████████████████████████████████████████████████████████████                                                    | 418/751 [00:02<00:01, 240.30it/s]

Evaluation:   0%|                                                                                                                               | 0/84 [00:00<?, ?it/s]

Evaluation:   1%|█▍                                                                                                                     | 1/84 [00:00<00:18,  4.51it/s]

Evaluation:  45%|████████████████████████████████████████████████████▉                                                                | 38/84 [00:00<00:00, 146.05it/s]

Evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 146.82it/s]

Training:  62%|████████████████████████████████████████████████████████████████████████▎                                            | 464/751 [00:03<00:02

>>> training loss: 0.0000, valid loss: 1.8550, valid f1 score: 0.4999



Training:  97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 729/751 [00:04<00:00, 241.69it/s]

Evaluation:   0%|                                                                                                                               | 0/84 [00:00<?, ?it/s]

Evaluation:   1%|█▍                                                                                                                     | 1/84 [00:00<00:19,  4.33it/s]

Evaluation:  44%|███████████████████████████████████████████████████▌                                                                 | 37/84 [00:00<00:00, 139.83it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:58<00:00,  5.87s/it]

>>> training loss: 0.0000, valid loss: 1.8713, valid f1 score: 0.5087


In [80]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Evaluation', total=len(test_dataloader))
    model.eval()
    test_preds = []
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}

            logits = model(**batch)[1]

            test_preds.append(logits.argmax(dim=-1).detach().cpu())
    
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]

    test_df = pd.read_csv(config['test_file_path'], sep=',')
    test_df.insert(1, column='label', value=test_preds)
    test_df.drop(columns=['sentence'], inplace=True)
    test_df.to_csv('submission.csv', index=False, encoding='utf8')